# Cross Validation Code Along

In [ ]:
from seaborn import get_dataset_names, load_dataset
import numpy as np
import pandas as pd

In [ ]:
# Run without changes
diamonds = load_dataset('diamonds')
diamonds.head()

In [ ]:
# Run without changes
from seaborn import get_dataset_names, load_dataset
from sklearn.preprocessing import LabelBinarizer

diamonds = load_dataset('diamonds')

# 21551 diamonds are ideal, so almost half of the 53939
# We will change this into an ideal/not ideal binary problem

def make_ideal_not_ideal(element):
    if element == "Ideal":
        return 1
    else:
        return 0

lb = LabelBinarizer()
diamonds['cut'] = diamonds['cut'].apply(make_ideal_not_ideal)
diamonds.head()

In [ ]:
diamonds['cut'].value_counts()

# Train Test Split

We have a healthy dataset here, as far as records go.  There are almost 54,000 records.

In the cell below, split the data with a test size of 20%.   


In [ ]:
from sklearn.model_selection import train_test_split


# Cross Validation

Now that we have split off the test set, we can experiment with the train set. We will use cross-validation to shuffle the training data, as shown in the diagram below.

![cross_val](images/grid_search_cross_validation.png)

Let's build a **First Simple Model** using a `DummyClassifier.`  

The DummyClassifier takes an argument `strategy`, which dictates what type of predictions it makes.  Set `strategy` equal to `most_frequent`. Based on our knowledge of the class balance, the DummyClassifer should always predict `0`, i.e. `not_ideal`.

Put the dummy classifier into a `cross_validate` function, and calculate the mean test scores.

In [ ]:
from sklearn.dummy import DummyClassifier

# your code here


In [ ]:
from sklearn.model_selection import cross_validate

# your code here

You should have returned a mean test score of .60.  What does that .60 represent?

#your answer here

# Logistic Regression

Instantiate a logistic regression classifier with the default parameters plus `random_state=42`.  Feed that into the `cross_validate` function. Make sure to set `return_train_score=True`.  Print out the mean train and test scores.

**hint** Logistic Regression will break with categoricals.  Feed in only continuous variables.

In [ ]:
# your code here

Answer the following questions about the above results:

Does logistic regression perform better than the dummy classifier?

Is the model high or low bias? What about high or low variance?  
Is the model over or under fit? 


Use the `cross_val_predict` function from sklearn to investigate the actual predictions across the different folds:  [cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)

Put those predictions into a confusion matrix using sklearn's. `confusion_matrix` method.


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# your code here

# Scaling

Remember that scaling is important when regularization is being peformed. If you look at the Logistic Regression docstring, you will see that `l2` regularization is being performed by default.  Let's see the effect scaling the data has on the scores.

In [ ]:
# instantiate a standard scalar, scale the data and feed it into cross_validate
from sklearn.preprocessing import StandardScaler

# print out mean train and test scores

Describe where data leakage occured in the above piece of code.

To ensure there is no leakage, we will implement KFold cross_validation.  

In [ ]:
from sklearn.model_selection import KFold

kf = KFold()

train_score = []
vald_score = []

for train_ind, val_ind in kf.split(X_train_continuous, y_train):
    # use train_ind and iloc to isolate the training records associated with each fold
    
    # use val_ind to isolate validation records
    
    # instantiate a scalar and fit it on the train. 
    
    # transform the train and val records with the scalar
    
    # instantiate a logistic regression model
    
    # fit logistic regression on train set.
    
    # score on both train and val and append to train/val_score lists
    


The above code is hard! KFolds helps us understand what is happening under the hood.  When we get to pipelines, however, much of this can be done in a few lines of code. 